![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# 使用watsonx, 和 LangChain 对语言模型进行一系列调用

#### 声明

- 仅使用在watsonx可用的项目和空间。


## Notebook内容

这篇notebook包含步骤和代码，支持通过watsonx模型和langchain集成来演示Simple Sequential Chain。对于Python有一定的熟练度会有帮助。这篇notebook使用Pythong 3.10。


## 学习目标

这篇notebook的目标是演示串联`google/flan-ul2`和`google/flan-t5-xxl`模型来生成一系列关于给定主题的随机问题和该问题的答案，让用户熟悉LangChain框架，使用WatsonxLLM, 简单链 (LLMChain) 和扩展链 (SimpleSequentialChain)。


## 内容

这篇notebook包含下列部分：

- [配置](#setup)
- [watsonx基础模型](#models)
- [LangChain集成](#watsonxllm)
- [Simple Sequential Chain实验](#experiment)
- [总结](#summary)

<a id="setup"></a>
## 配置环境

在开始使用notebook的示例代码之前，需要完成下列配置任务：

-  关联一个 <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) 实例</a> instance (关于如何创建实例的信息 <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### 安装并导入 `datasets` 和依赖包

In [2]:
!pip install "ibm-watson-machine-learning>=1.0.327" | tail -n 1
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install langchain | tail -n 1

### 定义 WML credentials
这个单元定义使用watsonx基础模型推理所需要的WML credentials。

**操作:** 提供IBM Cloud用户API Key，详情参考
[documentation](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui).

In [3]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

Please enter your WML api key (hit enter): ········


### 定义 project id
基础模型需要project id，用于给模型调用提供上下文信息。我们将从运行此笔记本的项目中获取id。否则，请另外提供project id。

In [4]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="models"></a>
## `watsonx.ai`的基础模型

#### 列出可用的模型

所有可用的模型都显示在ModelTypes类别下。
更多信息可参考 [documentation](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html#ibm_watson_machine_learning.foundation_models.utils.enums.ModelTypes).

In [5]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT']


需要定义用于模型推理的 `model_id`:

In [6]:
model_id_1 = ModelTypes.FLAN_UL2
model_id_2 = ModelTypes.FLAN_T5_XXL

### 定义模型参数

可能需要调整模型 `parameters` 用于不同的模型和任务, 具体可参考 `GenTextParamsMetaNames` 类别.

**操作:** 详细API可参考 [documentation](https://ibm.github.io/watson-machine-learning-sdk/).

In [7]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

### 初始化模型
用之前定义的参数初始化 `Model` 类。

In [8]:
from ibm_watson_machine_learning.foundation_models import Model

flan_ul2_model = Model(
    model_id=model_id_1, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

flan_t5_model = Model(
    model_id=model_id_2,
    credentials=credentials,
    project_id=project_id)

<a id="watsonxllm"></a>
## LangChain集成

`WatsonxLLM` 封装watsonx.ai模型，提供模型的链集成。

**操作:** 更多关于`CustomLLM`可参考 [LangChain documentation](https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm) 


### 初始化 `WatsonxLLM` 类

In [9]:
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

flan_ul2_llm = WatsonxLLM(model=flan_ul2_model)
flan_t5_llm = WatsonxLLM(model=flan_t5_model)

**提示:** 如果需要将watsonx.ai模型与LangChain的Chain接口结合使用，调用`model.to_langchain()`方法。会返回 `WatsonxLLM`类，与LangChain CustomLLM 要求兼容。

In [10]:
flan_ul2_model.to_langchain()

WatsonxLLM(model=<ibm_watson_machine_learning.foundation_models.model.Model object at 0x7f1f2dc05b70>)

使用`dict()`方法，打印所有WatsonxLLM对象中定义的数据。

In [11]:
flan_ul2_llm.dict()

{'model_id': 'google/flan-ul2',
 'params': {'decoding_method': <DecodingMethods.SAMPLE: 'sample'>,
  'max_new_tokens': 100,
  'min_new_tokens': 1,
  'temperature': 0.5,
  'top_k': 50,
  'top_p': 1},
 'project_id': '76a44507-bf62-4637-9115-fd3ab710edca',
 'space_id': None,
 '_type': 'IBM watsonx.ai'}

<a id="experiment"></a>
## Simple Sequential Chain实验
最简单的sequential chain类型是 `SimpleSequentialChain`, 其中每个步骤都有一个输入和输出，上一个步骤的输出会作为下一个步骤的输入。

该实验将包括生成一个关于任何主题的随机问题并回答该问题。

`PromptTemplate`对象通过用户输入、附加非静态数据和固定模板字符串的组合来帮助生成提示。

在我们的例子中，我们想要创建两个`PromptTemplate`对象，分别用于负责创建一个随机问题并回答它。

In [12]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="Generate a random question about {topic}: Question: "
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

我们想要使用`LLMChain`链围绕语言增加功能。
`prompt_to_flan_ul2` 链规定了提示模板的格式，其任务是生成随机问题，将字符串传递给LLM并返回LLM输出。

**提示:** 如果需要将watsonx.ai模型与LangChain的Chain接口结合使用，调用`model.to_langchain()`方法。会返回 `WatsonxLLM`类，与LangChain CustomLLM 要求兼容。

In [13]:
from langchain.chains import LLMChain

prompt_to_flan_ul2 = LLMChain(llm=flan_ul2_model.to_langchain(), prompt=prompt_1)

`flan_to_t5` 链规定了提示模版的格式，其任务是回答我们从`prompt_to_flan_ul2`链得到的问题, 将字符串传递给LLM并返回LLM输出。

In [14]:
flan_to_t5 = LLMChain(llm=flan_t5_model.to_langchain(), prompt=prompt_2)

这是完整的链，我们运行 `prompt_to_flan_ul2` 和 `flan_to_t5` 按序列组成的链.

In [15]:
from langchain.chains import SimpleSequentialChain

qa = SimpleSequentialChain(chains=[prompt_to_flan_ul2, flan_to_t5], verbose=True)

生成关于某个话题的随机问题并回答。

In [16]:
qa.run('car rental')



> Entering new SimpleSequentialChain chain...
What is the most common car rental agency in the US?
alamo

> Finished chain.


'alamo'

## 生成中文话题的随机问题并回答

In [150]:
parameters_pt1 = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.STOP_SEQUENCES: ['\n'],
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1
}

parameters_pt2 = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1
}

llama2_model_pt1 = Model(
    model_id=ModelTypes.LLAMA_2_70B_CHAT, 
    params=parameters_pt1, 
    credentials=credentials,
    project_id=project_id)

llama2_model_pt2 = Model(
    model_id=ModelTypes.LLAMA_2_70B_CHAT, 
    params=parameters_pt2, 
    credentials=credentials,
    project_id=project_id)

In [151]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="""生成1个与{topic}相关的问题：
    问题：
    """
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="""用1句话回答下列问题：
    问题：{question}
    答案： """
)

In [152]:
prompt_to_llama2 = LLMChain(llm=llama2_model_pt1.to_langchain(), prompt=prompt_1)
llama2_to_llama2 = LLMChain(llm=llama2_model_pt2.to_langchain(), prompt=prompt_2)

In [153]:
qa = SimpleSequentialChain(chains=[prompt_to_llama2, llama2_to_llama2], verbose=True)

In [157]:
qa.run('租车')



> Entering new SimpleSequentialChain chain...
我想租一辆车，但是我不知道哪些租车公司可以在我所在的城市提供服务，也不知道它们的价格和质量是怎样的。

可以在网上搜索“租车公司 + 你所在的城市”，或者咨询当地旅游信息中心或朋友圈，了解不同租车公司的价格和质量，并且可以预订在线或者通过电话

> Finished chain.


'可以在网上搜索“租车公司 + 你所在的城市”，或者咨询当地旅游信息中心或朋友圈，了解不同租车公司的价格和质量，并且可以预订在线或者通过电话'

<a id="summary"></a>
## 总结和下一步

 你顺利地完成了这篇notebook!
 
 你学习到如何通过custom llm WastonxLLM使用Simple Squential Chain。
 
Check out our _[Online Documentation](https://ibm.github.io/watson-machine-learning-sdk/samples.html)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 
 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.